<a href="https://colab.research.google.com/github/inyminymoe/Data-Analysis-with-Open-Source/blob/main/%EC%98%A4%ED%94%88%EC%86%8C%EC%8A%A4%EA%B8%B0%EB%B0%98%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D(%EC%A4%91%EA%B0%84%EA%B3%BC%EC%A0%9C%EB%AC%BC)_%EA%B0%95%EC%9D%B8%EC%98%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

API 호출

In [4]:
# 설치 및 import
!pip install pandas requests

import requests
import pandas as pd

In [6]:
# API 설정
api_key = '7657646572696e7931313561474d6b41'
year, month = 2024, 12
url = 'http://openapi.seoul.go.kr:8088/7657646572696e7931313561474d6b41/json/energyUseDataStatInfo/1/5/2024/12/'

# API 호출
response = requests.get(url)

# 데이터 출력
print('URL:', url )
print('status_code:', response.status_code)
print('Content-Type:', response.headers.get('Content-Type'))

# 데이터(JSON) 확인
data = response.json()
print('data:', data)
rows = data['energyUseDataStatInfo']['row']

# 호출 성공/실패 출력
try:
    response.raise_for_status()
    print('API 호출 성공')

except requests.exceptions.HTTPError as e:
    print('API 호출 실패:', e)

df = pd.DataFrame(rows)
print(df.head())

URL: http://openapi.seoul.go.kr:8088/7657646572696e7931313561474d6b41/json/energyUseDataStatInfo/1/5/2024/12/
status_code: 200
Content-Type: application/json;charset=UTF-8
data: {'energyUseDataStatInfo': {'list_total_count': 4110, 'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'}, 'row': [{'YEAR': '2024', 'MON': '12', 'HOSTCODE': '강남구', 'DONGCODE': '기타', 'MM_TYPE': '학교', 'TYPE_DIV': '단체', 'TYPE_DIV_CD': 'TB_GROUP', 'CNT': '1', 'EUS': '40385', 'EUS1': '38786', 'EUS2': '38383', 'ECO2_1_MATH_YEAR': '1599', 'ECO2_2_MATH_YEAR': '677.976', 'ECO2_1_MATH': '1800.5', 'ECO2_2_MATH': '763.412', 'ECO2_1': '1800.5', 'ECO2_2': '1800.5', 'E_TOE_YEAR': '0.36777', 'E_RATE_YEAR': '4.123', 'E_TOE_AVG': '0.414', 'E_RATE_AVG': '4.666', 'GUS': '3471', 'GUS1': '6841', 'GUS2': '5855', 'GCO2_1_MATH_YEAR': '-3370', 'GCO2_2_MATH_YEAR': '-7548.8', 'GCO2_1_MATH': '-2877', 'GCO2_2_MATH': '-6444.48', 'GCO2_1': '-2877', 'GCO2_2': '-5754', 'G_TOE_YEAR': '-3.515', 'G_RATE_YEAR': '-49.262', 'G_TOE_AVG': '-3.001', 

In [9]:
# API 및 기간 설정
api_key = '7657646572696e7931313561474d6b41'
endpoint = 'energyUseDataSummaryInfo'
YEARS = range(2015, 2025)
start, end = 1, 1000
data_list = []

def build_url(y:int, m:int,) -> str:
    return f"http://openapi.seoul.go.kr:8088/{API_KEY}/json/{endpoint}/{start}/{end}/{year}/{month:02d}"

def is_personal(row):
    if str(row.get("TYPE_DIV","")).strip() == "개인":
        return True
    cd = str(row.get("TYPE_DIV_CD", "")).upper()
    return("PERSON" in cd) or ("TB_PERSON" in cd)

# 개인 + 현년만 모으기
all_rows = []
for y in YEARS:
    for m in range(1, 12+1):
        r = requests.get(url_of(y, m), timeout=30)
        if r.status_code != 200:
            print(f'[{y}-{m:02d}] HTTP {r.status_code}')
            continue
        try:
            rows = r.json().get(endpoint, {}).get('row', []) or []
        except Exception:
            print(f'[{y}-{m:02d}] JSON 파싱 실패')
            continue
        personal = [row for row in rows if is_personal(row)]

        for row in personal:
            row['year'] = y
            row['month'] = m
        print(f'[{y}-{m:02d}] 개인 {len(personal)}건')
        all_rows.extend(personal)

print("\n총 수집 건수:", len(all_rows))
df = pd.DataFrame(all_rows)


NameError: name 'url_of' is not defined

In [10]:
import requests
import pandas as pd

# ---- 설정 ----
API_KEY = '7657646572696e7931313561474d6b41'
ENDPOINT = 'energyUseDataSummaryInfo'   # 개인/현년에 유리한 요약 엔드포인트
YEARS = range(2015, 2025)               # 2015~2024
START, END = 1, 1000

# ---- URL 만들기 ----
def url_of(y: int, m: int) -> str:
    return f"http://openapi.seoul.go.kr:8088/{API_KEY}/json/{ENDPOINT}/{START}/{END}/{y}/{m:02d}"

# ---- '개인' 판정 (간단/안전) ----
def is_personal(row: dict) -> bool:
    if str(row.get("TYPE_DIV", "")).strip() == "개인":
        return True
    cd = str(row.get("TYPE_DIV_CD", "")).upper()
    return ("PERSON" in cd) or ("TB_PERSON" in cd)

# ---- 수집 루프 ----
all_rows = []
for y in YEARS:
    for m in range(1, 13):
        resp = requests.get(url_of(y, m), timeout=30)
        if resp.status_code != 200:
            print(f"[{y}-{m:02d}] HTTP {resp.status_code}")
            continue
        try:
            rows = resp.json().get(ENDPOINT, {}).get("row", []) or []
        except Exception as e:
            print(f"[{y}-{m:02d}] JSON 파싱 실패: {type(e).__name__}")
            continue

        # 과제 조건: '개인'만
        personal = [r for r in rows if is_personal(r)]

        # 메타 컬럼
        for r in personal:
            r["year"] = y
            r["month"] = m

        print(f"[{y}-{m:02d}] 개인 {len(personal)}건")
        all_rows.extend(personal)

print("\n총 수집 건수:", len(all_rows))

# ---- DataFrame 변환 (2-1 캡처용) ----
df = pd.DataFrame(all_rows)
print("\n=== (2-1) DataFrame info ===")
df.info()
print("\n=== (2-1) Head ===")
display(df.head())

# ---- (2-2) 계절 컬럼 ----
def season(m: int) -> str:
    if 3 <= m <= 5:  return "봄"
    if 6 <= m <= 8:  return "여름"
    if 9 <= m <= 11: return "가을"
    return "겨울"  # 12,1,2

df["year"]  = pd.to_numeric(df.get("year"), errors="coerce").astype("Int64")
df["month"] = pd.to_numeric(df.get("month"), errors="coerce").astype("Int64")
df["season"] = df["month"].apply(lambda x: season(int(x)) if pd.notna(x) else None)

print("\n=== (2-2) year / month / season ===")
display(df[["year","month","season"]].head())


[2015-01] 개인 0건
[2015-02] 개인 0건
[2015-03] 개인 0건
[2015-04] 개인 0건
[2015-05] 개인 0건
[2015-06] 개인 0건
[2015-07] 개인 0건
[2015-08] 개인 0건
[2015-09] 개인 0건
[2015-10] 개인 0건
[2015-11] 개인 0건
[2015-12] 개인 0건
[2016-01] 개인 0건
[2016-02] 개인 0건
[2016-03] 개인 0건
[2016-04] 개인 0건
[2016-05] 개인 0건
[2016-06] 개인 0건
[2016-07] 개인 0건
[2016-08] 개인 0건
[2016-09] 개인 0건
[2016-10] 개인 0건
[2016-11] 개인 0건
[2016-12] 개인 0건
[2017-01] 개인 0건
[2017-02] 개인 0건
[2017-03] 개인 0건
[2017-04] 개인 0건
[2017-05] 개인 0건
[2017-06] 개인 0건
[2017-07] 개인 0건
[2017-08] 개인 0건
[2017-09] 개인 0건
[2017-10] 개인 0건
[2017-11] 개인 0건
[2017-12] 개인 0건
[2018-01] 개인 0건
[2018-02] 개인 0건
[2018-03] 개인 0건
[2018-04] 개인 0건
[2018-05] 개인 0건
[2018-06] 개인 0건
[2018-07] 개인 0건
[2018-08] 개인 0건
[2018-09] 개인 0건
[2018-10] 개인 0건
[2018-11] 개인 0건
[2018-12] 개인 0건
[2019-01] 개인 0건
[2019-02] 개인 0건
[2019-03] 개인 0건
[2019-04] 개인 0건
[2019-05] 개인 0건
[2019-06] 개인 0건
[2019-07] 개인 0건
[2019-08] 개인 0건
[2019-09] 개인 0건
[2019-10] 개인 0건
[2019-11] 개인 0건
[2019-12] 개인 0건
[2020-01] 개인 0건
[2020-02] 개인 0건
[2020-03

""


TypeError: data type 'Int64' not understood